# Chemical Intervention on Rat

This notebook is an exercise relying on the ISA-API functionality for building an experiment description based on the experimental design.

### Objectives
- Identify concepts from the experimental design
- Represent the experimental design concepts using the ISA-API model  
- Produce ISA metadata for the experiment

### References
- [Documentation about the ISA-API](https://isatools.readthedocs.io), and more specifically:
    - [documentation about study-design driven creation of ISA content](https://isatools.readthedocs.io/en/latest/studydesigncreation.html)
    - [documentation of main ISA objects](https://isatools.readthedocs.io/en/latest/creation.html#)


# Part 1

Read the following description of tan experiment and follow the steps below to identify the main elements from the experimental design.


Experiment Narrative:
---------------------

*Male Fisher F344 rats* purchased from Charles River were treated with *3 commonly used painkillers*, namely *acetylsalicilic acid, acetaminophen and ibuprofen*, at 2 distinct dose levels, delivered *per os*.
*Equal number of animals (n=5) were allocated to each of the group defined by a compound, dose level and duration post exposure combination.*
Following sacrifice performed by cervical dislocation and exsanguination preceded by anesthesia (ketamine and xylazine solution), *blood and kidney specimens were collected*.
*Total RNA were extracted from Kidney samples and mRNA sequencing (transcription profiling) was performed using paired-end libraries on Illumina sequencing platform using an Illumina HiSeq 2000 instrument*.
Blood samples were collected at sacrifice time and immediately placed in precooled 60 percent methanol ammonium bicarbonate buffer to quench cellular metabolism. Blood Metabolites were separated in water-soluble and lipophilic fractions. 
*Metabolite profiling was performed on the polar metabolite fraction only, using flow injection analysis (FIA) mass spectrometry on an Agilent 6550 iFunnel Q-TOF Mass Sprectrometry platform*. Each fraction was injected twice and data were acquired in both ionization modes (positive mode and negative mode).
Raw data files were saved in native instrument format and later converted to HUPO-PSI standard format for mass spectrometry.


## Experimental description with structured metadata

In order to describe the experiment, we will rely on the models defined by the ISA-API, and thus we need to following import statements:

In [64]:
from isatools.create.models import *
from isatools.model import *

## Identification of Variables, their levels and definition of the Treatment Plans

In the context of the above experiment, ** can you identify the independent variables and their associated levels**?.

You can define a new variable by relying in the ```StudyFactor``` object in this way:

```python
chemical_agent = StudyFactor(name="chemical agent")

```
or if you want to add the ontology term for such study factor, you can use the [EBI Ontology Lookup Service](https://www.ebi.ac.uk/ols/) to find a relevant term from the ChEBI ontology, and build the factor in this way:
    
```python
chemical_agent = StudyFactor(name="agent", factor_type=OntologyAnnotation(term="chemical entity", term_source="ChEBI", term_accession="http://purl.obolibrary.org/obo/CHEBI_24431"))
```    

Define the relevant ```StudyFactor```'s below and see if you can find ontology terms that are relevant to annotate them (e.g. you can try and find terms from the Experimental Factor Ontology or EFO):


In [65]:
### ANSWER

chemical_agent = StudyFactor(name="agent", factor_type=OntologyAnnotation(term="chemical entity", term_source="ChEBI", term_accession="http://purl.obolibrary.org/obo/CHEBI_24431"))
dose = StudyFactor(name="dose")
duration = StudyFactor(name="duration")


Next, we can use a ```TreatmentFactory``` to include the ```StudyFactor```'s and indicate their different factor levels, i.e. the values that these variables will assume according to the experiment description above.

You should create the treatment factory providing the different study factors:

```python
treatment_factory = TreatmentFactor(factors=[ ... here list the variables...])

```
and then for each factor, you can add their levels in the following way:

```python
treatment_factory.add_factor_value( ... factor variable ..., { ... list of strings with the names of the factor values... }
```


In [66]:
### ANSWER

treatment_factory = TreatmentFactory(factors=[chemical_agent, dose, duration])

treatment_factory.add_factor_value(chemical_agent, {'acetyl salicylic acid', 'acetaminophen', 'ibuprofen'})
treatment_factory.add_factor_value(dose, {'high dose', 'low dose'})
treatment_factory.add_factor_value(duration, {'t1', 't2', 't3'})


### Computing the Number of Unique Treatment Groups/Study Groups: 


As per the description above, the experiment follows a factorial design where all combinations of factor values are considered. We can build our treatment plan, or ```TreatmentSequence```, by relying on an utilty method that given the factors and their levels computes the full factorial design:



In [67]:
all_treatments = treatment_factory.compute_full_factorial_design()

How many treatment groups should have been identified? Check the number you considered matches with the answer of running the command below:

In [68]:
print('Number of study groups (treatment groups): {}'.format(len(all_treatments)))

Number of study groups (treatment groups): 18


We can now build a treatment plan, or ```TreatmentSequence```, by including all the treatments according to the factorial design:

In [69]:
treatment_sequence = TreatmentSequence(ranked_treatments=all_treatments)

Are study subjects exposed to a single intervention or to multiple intervention?
  


### Study group size

The following code builds a slider (relying on the ```ipywidgets``` library) for you to set the group size, please select the appropriate number according to the experiment description:

In [70]:
from ipywidgets import (IntSlider)
group_size = IntSlider(value=1, min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
group_size


The group size value you chose, and that is going to be used in the next section, is:

In [71]:
group_size.value

1

## Sample collection and assay plans

Given the group size selected above, we are now going to build the sample collection and assay plans based on the group size. Given this, can you say if the design is balanced or unbalanced?

In [72]:
plan = SampleAssayPlan(group_size=group_size.value)

Let's now build a ```dictionary``` with the sample collection plan: it should contain key:value pairs with the specimen or sample type as key and the number of samples collected for each type over the course of the study. Here is the code snippet that you should complete:

```python
sample_collection_plan = { "sample type 1": 0, "sample type 2": 1 }
```


In [73]:
### ANSWER
sample_collection_plan = {"kidney":1, "blood":1} 


Next, the following code will take the sample_collection_plan object that you built and include all the details in the sample_collection_plan object:

In [74]:
for sample_typ in sample_collection_plan:    
    plan.add_sample_type(sample_type)
    plan.add_sample_plan_record(sample_type,sample_collection_plan[key])


## Create an ISA experimental description based on the study design and the sampling plan:

In the following section, the task is to build ISA objects relying on the study design information we built above.


Let's first create an ```Investigation``` object to hold all the information about the experiment with this command, which also assigns an identifier for the investigation.

In [76]:
isa_investigation = Investigation(identifier='inv-dtp-exercise')

Now, let's create a study object using the sample and assay plan as well as the ```TreatmentSequence``` object we built before. For this, we will need an object of the ```IsaModelObjectFactory``` class provided in the ISA-API:

In [75]:
isa_object_factory = IsaModelObjectFactory(plan, treatment_sequence)
isa_study = isa_object_factory.create_study_from_plan()


Now, we can link the study to the investigation we created before:

isa_investigation.studies = [isa_study]


an set a name for the study file:

In [ ]:
isa_study.filename = '... complete here a filename for the study file...'

In [ ]:
## ANSWER

isa_study.filename = 's_study.txt'

### Study description (or abstract) and study design type


Can you set the study description relying in this code snippet?

```python
isa_study.description = "... here the text of the description..."
```


In [78]:
### ANSWER

isa_study.description = "*Male Fisher F344 rats* purchased from Charles River were treated with *3 commonly used painkillers*, namely *acetylsalicilic acid, acetaminophen and ibuprofen*, at 2 distinct dose levels, delivered *per os*. *Equal number of animals (n=5) were allocated to each of the group defined by a compound, dose level and duration post exposure combination.* Following sacrifice performed by cervical dislocation and exsanguination preceded by anesthesia (ketamine and xylazine solution), *blood and kidney specimens were collected*. *Total RNA were extracted from Kidney samples and mRNA sequencing (transcription profiling) was performed using paired-end libraries on Illumina sequencing platform using an Illumina HiSeq 2000 instrument*. Blood samples were collected at sacrifice time and immediately placed in precooled 60 percent methanol ammonium bicarbonate buffer to quench cellular metabolism. Blood Metabolites were separated in water-soluble and lipophilic fractions.*Metabolite profiling was performed on the polar metabolite fraction only, using flow injection analysis (FIA) mass spectrometry on an Agilent 6550 iFunnel Q-TOF Mass Sprectrometry platform*. Each fraction was injected twice and data were acquired in both ionization modes (positive mode and negative mode). Raw data files were saved in native instrument format and later converted to HUPO-PSI standard format for mass spectrometry."

Next, we would like to specify the type of study design (and we can set multiple values if necessary). Consider the following questions'intervention design' or an 'observation design'. Which one will you set?

*Check state of the Sample Assay Plan after entering sample planning information:*

In [79]:

dataframes = dumpdf(isa_investigation)
sample_table = next(iter(dataframes.values()))
show_grid(sample_table)

NameError: name 'dumpdf' is not defined

In [ ]:
print('Total rows generated: {}'.format(len(sample_table)))

# Saving key study design information as a JSON document.

This section is meant to show how to serialize key study design descriptors in a compact document serialized in format format. 
Why is this relevant? How would you use this feature. List 3 possible uses.

In [ ]:
import json
from isatools.create.models import SampleAssayPlanEncoder
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

## Documenting the Assay and Data Acquisition Plans:

From the textual description, identify the Response variables / Dependent Variables

### Binding assay technology types to sample type: 
Check against ISA configuration to obtain vetted values for Measurement Type and Technology type.
An ISA configuration can be accessed from: https://github.com/ISA-tools/Configuration-Files/tree/master/isaconfig-default_v2015-07-02

In [ ]:
assay_types = set()

assay_type1 = AssayType(measurement_type='transcription profiling', technology_type='nucleotide sequencing')
assay_type2 = AssayType(measurement_type='metabolite profiling', technology_type='mass spectrometry')

assay_types.add(assay_type1)
assay_types.add(assay_type2)
for x in assay_types:
        print(x.measurement_type.term," using ", x.technology_type.term)


## Assay Specific Descriptors
Each data acquisition modality comes with its own set of parameters which may be used to capture the underlying workflow graph modifications. This section of the exercise aims to show how to deal with workflows specific to sequencing applications and mass spectrometry ones.

### Generate ISA model objects from the assay plan and render the assay table

In [ ]:
# Starting with DNA sequencing assay workflow: 
# Setting the type of library and the number of runs per sample.

# This bit is meant to introduce you to iwidgets, a library for ipython allowing user interaction (not implemented here)
ngs_technical_replicates = IntSlider(value=2, min=0, max=5, step=1, description='Technical repeats:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
print(ngs_technical_replicates.value)
ngs_technical_replicates



### Check state of Sample Assay Plan after entering assay plan information:

### Dealing with Next Generation Sequencing Data Acquisition Plan

In [ ]:
sequencing_instruments = set()
sequencing_instruments.add('Illumina HiSeq 2000') # check the experiment description to obtain value.

top_mods_seq = DNASeqAssayTopologyModifiers(technical_replicates=ngs_technical_replicates.value, instruments=sequencing_instruments,distinct_libraries=2)

print('Technical replicates: {}'.format(top_mods_seq.technical_replicates))

assay_type1.topology_modifiers = top_mods_seq

plan.add_assay_type(assay_type1)
# sample_type = plan.sample_types[]
for element in plan.sample_types:
    if element.value.term == 'kidney':
        print(element.value.term)
        plan.add_assay_plan_record(element.value.term, assay_type1)
        assay_plan = next(iter(plan.assay_plan))
    elif element.value.term == 'blood':
        print("not for this assay type")
    else:
        print(element.value.term)

# if sample_type not in plan.sample_types:
#     print(sample_type + " Not in sample types")
#else:    
#    plan.add_assay_plan_record(sample_type, assay_type1)  # check the experiment description to obtain value.
   

print('Added assay plan: {0} -> {1}/{2}'.format(assay_plan[0].value.term, assay_plan[1].measurement_type.term, assay_plan[1].technology_type.term))
if len(top_mods_seq.instruments) > 0:
    print('Instruments: {}'.format(list(top_mods_seq.instruments)))

In [ ]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

### Dealing with Mass Spectrometry Data Acquisition Plan

In [ ]:
chromatography_instruments = set()
ms_instruments = set()
injection_modes = set()
acquisition_modes = set()

ms_instruments.add('agilent') # check the experiment description to obtain value.
injection_modes.add('FIA')  # check the experiment description to obtain value.
acquisition_modes.add('positive') # check the experiment description to obtain value.
acquisition_modes.add('negative') # check the experiment description to obtain value.
ms_tech_rep = 1 # check the experiment description to obtain value.
top_mods_ms = MSAssayTopologyModifiers(technical_replicates=ms_tech_rep, injection_modes=injection_modes, acquisition_modes=acquisition_modes, instruments=ms_instruments, chromatography_instruments=chromatography_instruments)
assay_type2.topology_modifiers = top_mods_ms

if len(top_mods_ms.chromatography_instruments) > 0:
    print('Chromatography instruments: {}'.format(list(top_mods_ms.chromatography_instruments)))
else:
    print('no chromatography used or no information supplied')

if len(top_mods_ms.instruments) > 0:
    print('Data acquisition instruments: {}'.format(list(top_mods_ms.instruments)))    
if len(top_mods_ms.injection_modes) > 0:
    print('Injection modes: {}'.format(list(top_mods_ms.injection_modes)))
if len(top_mods_ms.acquisition_modes) > 0:
    print('Acquisition modes: {}'.format(list(top_mods_ms.acquisition_modes)))


plan.add_assay_type(assay_type2)
plan.add_assay_plan_record("blood", assay_type2)

assay_plan = next(iter(plan.assay_plan))



In [ ]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
isa_investigation.studies = [isa_object_factory.create_assays_from_plan()]
for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, len(assay.samples), len(assay.process_sequence), len(assay.data_files)))
dataframes = dumpdf(isa_investigation)

In [ ]:
show_grid(dataframes[next(iter(dataframes.keys()))])

In [ ]:
show_grid(dataframes['a_blood_ms_FIA_positive_assay.txt'])

In [ ]:
show_grid(dataframes['a_blood_ms_FIA_negative_assay.txt'])